<a href="https://colab.research.google.com/github/pankajsingh09/Hands-on-Big-Data-Practices-with-PySpark-Spark-Tuning/blob/main/Advaced_RDD_with_Transformation_and_Actions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Advaced RDD with Transformation and Actions.

In [ ]:
!pip install pyspark
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("Advance RDD").setMaster("local[*]")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 199 kB 60.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=6cdb93f3bbd6bc89e7b25e5a9b72f3c206fdfe71e8f6e4eb9e386645e314f4b1
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [ ]:
sc = SparkContext(conf = conf)

In [ ]:
import random
list1 = random.sample(range(1,50),10)
list2 = random.sample(range(1,50),10)
print(list1)
print(list2)

[20, 26, 21, 11, 27, 38, 23, 18, 15, 22]
[40, 14, 26, 16, 3, 35, 11, 29, 30, 34]


In [ ]:
rdd1 = sc.parallelize(list1,4)
rdd2 = sc.parallelize(list2,4)
print(rdd1.glom().collect())
print(rdd2.glom().collect())

[[20, 26], [21, 11], [27, 38], [23, 18, 15, 22]]
[[40, 14], [26, 16], [3, 35], [11, 29, 30, 34]]


In [ ]:
rdd_union = rdd1.union(rdd2)
print(rdd_union.collect())

[20, 26, 21, 11, 27, 38, 23, 18, 15, 22, 40, 14, 26, 16, 3, 35, 11, 29, 30, 34]


In [ ]:
# Intersection
rdd_intersection = rdd1.intersection(rdd2)
print(rdd_intersection.collect())
print(rdd_intersection.glom().collect())
print(rdd_intersection.getNumPartitions())

[26, 11]
[[], [], [26], [11], [], [], [], []]
8


In [ ]:
# Find Empty Partition

count = 0
for item in rdd_intersection.glom().collect():
  if len(item) == 0:
    count = count + 1

count

6

In [ ]:
# Coalesce(numPartition)
rdd_intersection.coalesce(1).glom().collect()

[[26, 11]]

In [ ]:
# takeSample(withReplacement, num, [seed])
rdd1.takeSample(False ,5)

[22, 15, 20, 38, 18]

In [ ]:
# takeOrder(n, [ordering])
print(rdd1.takeOrdered(5))
print(rdd1.takeOrdered(5, key = lambda x: -x))

[11, 15, 18, 20, 21]
[38, 27, 26, 23, 22]


In [ ]:
# reduce
rdd1.reduce(lambda x,y :x*y)

16837475054400

In [ ]:
# reduceByKey
rdd_rbk = sc.parallelize([(1,4),(7,10),(5,7),(1,12),(7,12),(7,1),(9,1),(7,4)], 2)
print(rdd_rbk.glom().collect())
print(rdd_rbk.reduceByKey(lambda x,y :x+y).collect())

# user Friendly visualisation
import pandas as pd
counter = pd.DataFrame({'key': rdd_rbk.keys().collect(),
                        'values':rdd_rbk.values().collect()})
print(counter)

[[(1, 4), (7, 10), (5, 7), (1, 12)], [(7, 12), (7, 1), (9, 1), (7, 4)]]
[(1, 16), (7, 27), (5, 7), (9, 1)]
   key  values
0    1       4
1    7      10
2    5       7
3    1      12
4    7      12
5    7       1
6    9       1
7    7       4


In [ ]:
# sort By Key
print(rdd_rbk.reduceByKey(lambda x,y :x+y).sortByKey().collect())
#print in reverse order
print(rdd_rbk.reduceByKey(lambda x,y :x+y).sortByKey(False).collect())


[(1, 16), (5, 7), (7, 27), (9, 1)]
[(9, 1), (7, 27), (5, 7), (1, 16)]


In [ ]:
# countByKey
print(rdd_rbk.countByKey())
print(rdd_rbk.countByKey().items())
print(sorted(rdd_rbk.countByKey().items()))

defaultdict(<class 'int'>, {1: 2, 7: 4, 5: 1, 9: 1})
dict_items([(1, 2), (7, 4), (5, 1), (9, 1)])
[(1, 2), (5, 1), (7, 4), (9, 1)]


In [ ]:
# GroupByKey
rdd_group = rdd_rbk.groupByKey()
print(rdd_group.getNumPartitions())

print(rdd_group.collect())

for item in rdd_group.collect():
  print(item[0], [values for values in item[1] ])

2
[(1, <pyspark.resultiterable.ResultIterable object at 0x7fb55b3cd0d0>), (7, <pyspark.resultiterable.ResultIterable object at 0x7fb55b3cd050>), (5, <pyspark.resultiterable.ResultIterable object at 0x7fb55b3cd190>), (9, <pyspark.resultiterable.ResultIterable object at 0x7fb55b3cd210>)]
1 [4, 12]
7 [10, 12, 1, 4]
5 [7]
9 [1]


In [ ]:
# lookup(key)
rdd_rbk.lookup(7)

[10, 12, 1, 4]

In [ ]:
# cache:
# By default, each transformed RDD may be recomputed each time you run an action on it.
# However, you may also persist an RDD in memory using the persist (or cache) method,
# in which case Spark will keep the elements around on the cluster for much faster access the next time you query it.

rdd_rbk.persist() # OR rdd_Rbk.cache()

ParallelCollectionRDD[22] at readRDDFromFile at PythonRDD.scala:274

In [ ]:
# Persistence (https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence)
from pyspark import StorageLevel
rdd1.persist(StorageLevel.MEMORY_AND_DISK)

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274